In [1]:
import numpy as np
import os
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt

classes_list = ['sad','friendly','trap','photo','map','bright','deaf','away','help','I','meet','my','name']
num_of_videos = 30
sequence_len = 20
image_height = 240
image_width = 320

In [2]:
mp_holistic = mp.solutions.holistic      #Holistic model (detects)
mp_drawing = mp.solutions.drawing_utils  #drawing utilities   (draws)
mp_drawing_styles = mp.solutions.drawing_styles

holistic = mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5)


def draw_landmarks(image,results):
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(101,111,10),thickness=1,circle_radius=1),
                              mp_drawing.DrawingSpec(color=(80,256,121),thickness=1,circle_radius=1)
                             )
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(101,111,10),thickness=1,circle_radius=1),
                              mp_drawing.DrawingSpec(color=(80,256,121),thickness=1,circle_radius=1)
                             )
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(101,111,10),thickness=1,circle_radius=1),
                              mp_drawing.DrawingSpec(color=(80,256,121),thickness=1,circle_radius=1)
                             )

In [3]:
def save_video(frames, output_path):
    height, width, _ = frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, 30.0, (width, height))
    for frame in frames:
        out.write(frame)
    out.release()

In [4]:
checker = np.zeros(len(classes_list)).astype(np.int32)

In [9]:
os.makedirs('datasets',exist_ok=True)
os.makedirs('datasets/original',exist_ok=True)

cap = cv2.VideoCapture(0)

sign_num = 0

while True:
    while True:
        ret,frame = cap.read()
        if checker[sign_num] == True:
            done = 'done'
        else:
            done = 'not done'
            
        frame_rgb = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        results = holistic.process(frame_rgb)
        draw_landmarks(frame,results)
        cv2.putText(frame,f'{done} {sign_num+1}.{classes_list[sign_num]}',(120,200),cv2.FONT_HERSHEY_SIMPLEX,1.5,(255,60,255),3,cv2.LINE_AA)
        cv2.imshow('feed',frame)
        
        ch = cv2.waitKey(10)
        if ch == ord('q'):
            break
        if ch == ord('z') and sign_num > 0:
            sign_num-=1
        if ch == ord('x') and sign_num < len(classes_list)-1:
            sign_num+=1
        
    os.makedirs(f'datasets/original/{classes_list[sign_num]}',exist_ok=True)  
    
    for video_num in range(num_of_videos): 
        frames = []
        count = 0
        while(True):
            ret,frame = cap.read()
           
            if video_num == 0 and count == 0:
                cv2.putText(frame,f'Starting for...{classes_list[sign_num]}',(60,200),cv2.FONT_HERSHEY_SIMPLEX,1.5,(0,60,255),3,cv2.LINE_AA)
                cv2.imshow('feed',frame)
                cv2.waitKey(3000)
            else:
                frames.append(frame.copy())
                frame_rgb = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
                results = holistic.process(frame_rgb)

                draw_landmarks(frame,results)
                cv2.putText(frame,f'{classes_list[sign_num]}/{video_num+1}/{count}',(40,50),cv2.FONT_HERSHEY_SIMPLEX,1.5,(0,255,0),3,cv2.LINE_AA)
                cv2.imshow('feed',frame)
                if count == sequence_len:
                    cv2.waitKey(2000)
                    break

            count+=1
            if cv2.waitKey(1) == ord('q'):
                break
            
        
        save_video(frames,f'datasets/original/{classes_list[sign_num]}/{video_num}.mp4')
    
    checker[sign_num] = True
        
        
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [10]:
cap.release()
cv2.destroyAllWindows()